# Sea Ice Concentration CDR Reformatting

The goal of this code is to take the Sea Ice Concentration CDR NetCDF (.nc) gridded data, convert it to a single sea ice extent value for each day of a year, and output it to a .csv file. After running this code, you will have a csv file called CDR_daily_tot_extent.csv that contains sea ice extent for each day for a specified year. This file will be saved to the same location as the data are downloaded to. To begin, download the time period of the Final CDR (G02202) and the Near-Real-Time CDR (ID G10016) data set from NSIDC that you are interested in. 

**Final CDR Data File Path:**
ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/daily/

**Near-Real-Time CDR File Path:**
ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G10016/north/daily/


This can be accomplished with a variety of methods, but I personally used FileZilla to batch download the daily data, however the NSIDC website outlines a few other batch downloaders that can be used. 

The data must all be contained in the same directory. You will set the 'path' variable in this code to the directory where all of the data is stored. The path variable will be in format of “C:/Users/mypath/data/” as seen in the next code box. My data and python notebooks are all stored in a file folder on my desktop named '2020' with an additional imbedded directory exclusively for the .nc data named '2020'. These naming practices **do not** need to be the same for your path.

In addition, an extra file called 'psn25area_v3.csv' csv file used in this code will need to be downloaded and saved to the same working directory you use in this code. You can download that file from this location: https://nsidc.org/sites/nsidc.org/files/files/noaa/g02202/psn25area_v3.csv

After the code has run successfully, check the csv file to ensure it was created correctly. The naming practices in the output csv file will need to follow those of the compiling code which use variables 'yyyymmdd' and 'extent' as the two column names. Again, these can be renamed at the users discretion, but you will need to ensure all codes are updated to reflect any changes in naming.

Special thank you to Ann Windnagel for contributing to the construction of this code. 

In [5]:
import glob
import os
import os.path as op
import csv
import numpy as np
import numpy.ma as ma
from netCDF4 import Dataset
from datetime import datetime, timedelta, date
import pandas as pd

# Change path to where data and code reside
HOME = op.join(op.expanduser('~'))
path = "/Users/Michaela/Downloads/data/all_data"
os.chdir(path)


In [8]:
# Open CDR NetCDF file
files=(glob.glob('*.nc'))
files.sort()

tot_extent=[]
with open('CDR_daily_tot_extent.csv','w',newline='') as f1:
    writer=csv.writer(f1, delimiter=',')
    writer.writerow({'mmddyyyy', 'extent'})
    for f in files:
        dataset = Dataset(f)
       
        # Open csv file of pixel area size
        pixel_area_fn = 'psn25area_v3.csv'
        pixel_area = np.loadtxt(fname = pixel_area_fn, delimiter=",")
       
        # Pull out the time variable into a numpy array
        file_time = dataset.variables['time']
        file_time_np = np.array(file_time)
        # Convert the time that's in "days since 1601-01-01" to a human readable date
        # This is the "days since" part
        start = date(1601,1,1)
        
        # Create a time delta object from the number of days.
        # This may work for floats in general, but using integers
        # is more precise so convert your days to an int (e.g. int(file_time_np[0]))
        delta = timedelta(int(file_time_np[0]))
        
        # Add the specified number of days to 1601
        human_date = start + delta
        strp=datetime.strftime(human_date, '%m/%d/%Y')
        #srtd=sorted(strp)
        #print(strp)
    # Scale the pixel area file by 1000
        pixel_area_scaled = pixel_area/1000
        seaice_conc = dataset.variables['seaice_conc_cdr']
        seaice_conc_np = np.array(seaice_conc)
        # convert seaice_conc_np to a 2d array so it's easier to work with
        seaice_conc_np_2d = np.copy(seaice_conc_np.squeeze())
        
        # Find all the spots where the sea ice conc is between 0 and 1 because I need to be able to scale just these data
        ans = np.where(np.logical_and(np.greater(seaice_conc_np_2d,0.0),np.less_equal(seaice_conc_np_2d,1.0)))
        
        # scale the concentration data by multipling the concentration by 100 to get percent
        seaice_conc_np_2d_percent = np.copy(seaice_conc_np_2d)
        seaice_conc_np_2d_percent[ans] = seaice_conc_np_2d_percent[ans]*100
        
        # Create mask to filter out the flag values from the computation and concentration equal to or less than 15%
        # Flag values are 252, 253, 254, 255; Note: not removing 251 because that's the pole hole and it is
        # included in the extent total
        # This line removes flag values and concentration <= 15%
        flag_mask = ma.masked_outside(seaice_conc_np_2d_percent, 15, 251)
        
        # Now need to set all non-masked values to 1
        non_mask = ma.nonzero(flag_mask)
        flag_mask[non_mask] = 1.0
        
        # Apply the pixel area array to the masked sea ice
        sea_ice_extent_raster = flag_mask * pixel_area_scaled
        cdr_seaice_exent_total = sea_ice_extent_raster.sum()
        #print(cdr_seaice_exent_total, 'Million sq km')
        
        rows=[human_date,cdr_seaice_exent_total]
        tot_extent.append(cdr_seaice_exent_total)
       
        writer.writerow(rows)


The code above will write the csv file in the order of whatever file it first sees and then continue down the list one after another. To fix this, the next code box will read the newly created csv file, create a pandas dataframe from it, sort the data by date, then write that dataframe to a csv file. This ensures that when this data set is eventually plotted using matplotlib, there will be no inconsistencies in the output graph. 

In [9]:
cdr= pd.read_csv("CDR_daily_tot_extent.csv",parse_dates=True)
cdr = cdr.sort_values(by='mmddyyyy')
cdr.to_csv('CDR_daily_tot_extent.csv', index=False)